In [1]:
# default_exp strategy

# strategy

> A backtesting strategy.

In [2]:
#hide
from nbdev.showdoc import *

In [3]:
#export
# import modin.pandas as pd
import pandas as pd
import numpy as np
def isbday(date):
    return bool(len(pd.bdate_range(date, date)))

In [4]:
#export
class Strategy:
    def __init__(df=None, time=None, code=None, number=None):
        #应该具有包容性，可以传入数组，也可以传入df，也可以什么都不传
        if df:
            self.df=df
            self.time=df.index
            self.code=df['code']
            self.number=df['number']
            self.is_buy=df['is_buy']
        elif time and code and number and is_buy:
            self.code=pd.Series(code, name='code')
            self.number=pd.Series(number, name='number') 
            self.is_buy=pd.Series(is_buy, name='is_buy') 
            self.df = pd.concat([self.code, self.number, self.is_buy])
            self.df.index=pd.DatetimeIndex(time, name='time')
        else: # 什么数据都没有
            #TODO 创建一个空的有标签的df
            self.df=pd.Dataframe()
            self.time=df.index
            self.code=df['code']
            self.number=df['number']
            self.is_buy=df['is_buy']
            self.df.index=pd.DatetimeIndex(time, name='time')
    def beg(self):
        pass

In [5]:
pr = pd.read_csv('../price_2005_2020.csv', index_col='time', parse_dates=['time'])
pr.tail()

,code,open,close,high,low
time,,,,,
2020-12-25,689009.XSHG,72.01,75.10,75.50,67.61
2020-12-28,689009.XSHG,74.82,80.10,83.20,74.32
2020-12-29,689009.XSHG,81.38,77.46,83.80,76.55
2020-12-30,689009.XSHG,78.13,80.91,83.49,76.21
2020-12-31,689009.XSHG,82.00,85.82,88.00,80.09


In [6]:
#TODO pandas中如何根据两列进行定位？？？
# u4，u2只有65535不够。这里因为有000001这样的代码，如果使用纯数字，就要对1进行数位补齐，或者对原始数据进行处理。
code = np.array(['600211.XSHG']*2)
# i4，正数买入，负数卖出
number = [100, -100] 
# 暂时均按开盘价处理
time = pd.DatetimeIndex(['2005-03-18', '2020-12-17'])
df = pd.DataFrame({'code':code, 'number':number}, index=time)
df['cash']=None
df

,code,number,cash
2005-03-18,600211.XSHG,100,None
2020-12-17,600211.XSHG,-100,None


In [7]:
# %time a = pr.loc['2005-01-04'][pr.loc['2005-01-04']['code']=='600211.XSHG']['open'][0]
# a

CPU times: user 41.1 ms, sys: 0 ns, total: 41.1 ms
Wall time: 42.8 ms


2.51

In [15]:
# df.iat[0, 1] = 100
# df.iat[1, 1] = -100
# df.loc[[df.index[0]]] 选取第一行
for i in range(2):
    date = df.index[i]
    code = df.iat[i, 0]
    number = df.iat[i, 1]
    cash = df.iat[i, 2]
    date_lines = pr.loc[date]
    code_line = date_lines[date_lines['code']==code]
    _open = code_line['open'][0]
    print(_open)
    df.iat[i, 2] = -number * _open
df

2.64
62.95


,code,number,cash
2005-03-18,600211.XSHG,100,-264
2020-12-17,600211.XSHG,-100,6295


In [9]:
pr.loc[df.index[1]]

,code,open,close,high,low
time,,,,,
2020-12-17,000001.XSHE,18.91,18.95,19.07,18.60
2020-12-17,000002.XSHE,28.61,28.84,28.93,28.30
2020-12-17,000004.XSHE,23.17,23.18,23.42,22.67
2020-12-17,000005.XSHE,2.53,2.55,2.55,2.50
2020-12-17,000006.XSHE,5.49,5.52,5.52,5.36
...,...,...,...,...,...
2020-12-17,688608.XSHG,369.00,372.00,379.88,326.50
2020-12-17,688777.XSHG,98.00,98.92,102.01,97.50
2020-12-17,688788.XSHG,193.73,191.85,195.50,186.86
